In [102]:
import numpy as np
import pandas as pd
import plotly
# Loading csv file into a pandas DataFrame
df = pd.read_csv('TLC_New_Driver_App.csv') 
df = df.rename(columns={'App Date': 'App Date (M/D/Y)'}) #Changing column name to include (m/d/Y)
print(df.info())#Some info about the dataframe
df.head()#first 5 rows

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7108 entries, 0 to 7107
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   App No                   7108 non-null   int64 
 1   Type                     7108 non-null   object
 2   App Date (M/D/Y)         7108 non-null   object
 3   Status                   7108 non-null   object
 4   FRU Interview Scheduled  7108 non-null   object
 5   Drug Test                7108 non-null   object
 6   WAV Course               7108 non-null   object
 7   Defensive Driving        7108 non-null   object
 8   Driver Exam              7108 non-null   object
 9   Medical Clearance Form   7108 non-null   object
 10  Other Requirements       7108 non-null   object
 11  Last Updated             7108 non-null   object
dtypes: int64(1), object(11)
memory usage: 666.5+ KB
None


,App No,Type,App Date (M/D/Y),Status,FRU Interview Scheduled,Drug Test,WAV Course,Defensive Driving,Driver Exam,Medical Clearance Form,Other Requirements,Last Updated
0,5990450,HDR,02/17/2022,Incomplete,Not Applicable,Complete,Complete,Complete,Needed,Complete,Not Applicable,04/01/2022 05:00:03 PM
1,5961459,HDR,02/26/2020,Incomplete,Not Applicable,Needed,Needed,Complete,Needed,Needed,Copy of DMV license needed,04/01/2022 05:00:03 PM
2,5989657,HDR,02/02/2022,Incomplete,Not Applicable,Complete,Complete,Complete,Needed,Complete,Not Applicable,04/01/2022 05:00:03 PM
3,5992972,HDR,03/29/2022,Incomplete,Not Applicable,Needed,Needed,Needed,Needed,Needed,Fingerprints needed; Copy of DMV license needed,04/01/2022 05:00:03 PM
4,5990896,HDR,02/24/2022,Incomplete,Not Applicable,Needed,Needed,Needed,Needed,Needed,Fingerprints needed; Copy of DMV license needed,04/01/2022 05:00:03 PM


In [126]:
df['App Date (M/D/Y)'] = pd.to_datetime(df['App Date (M/D/Y)'],format = '%m/%d/%Y')#Converts App Date column into a datetime instance column
# type(df['App Date (M/D/Y)'])
df = df.sort_values(by = 'App Date (M/D/Y)') #Sorts DF by app Date (can only be done if converted to datetime instances)
df.head()

,App No,Type,App Date (M/D/Y),Status,FRU Interview Scheduled,Drug Test,WAV Course,Defensive Driving,Driver Exam,Medical Clearance Form,Other Requirements,Last Updated
411,5921772,PDR,1997-04-28,Incomplete,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Fingerprints & other items needed. Visit www.n...,04/01/2022 05:00:03 PM
6603,5921774,PDR,1997-04-28,Incomplete,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Fingerprints & other items needed. Visit www.n...,04/01/2022 05:00:03 PM
2880,5921773,PDR,1997-04-28,Incomplete,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Fingerprints & other items needed. Visit www.n...,04/01/2022 05:00:03 PM
795,5735329,HDR,2016-10-17,Incomplete,Not Applicable,Complete,Complete,Complete,Needed,Complete,Copy of DMV license needed,04/01/2022 05:00:03 PM
1142,5775919,HDR,2017-03-22,Incomplete,Not Applicable,Complete,Complete,Complete,Needed,Complete,Not Applicable,04/01/2022 05:00:03 PM


In [135]:
group2 = df.groupby(['Driver Exam',pd.Grouper(key = 'App Date (M/D/Y)', freq = 'Y')])
df3 = group2.count().reset_index()
df3
# for key, values in group2:
#     print(key)
#     print('-'*70)
#     print(values)
#     print('\n')
# df3 = group.agg({'App No':'count'})#.reset_index()
# df3

,Driver Exam,App Date (M/D/Y),App No,Type,Status,FRU Interview Scheduled,Drug Test,WAV Course,Defensive Driving,Medical Clearance Form,Other Requirements,Last Updated
0,Complete,2017-12-31,1,1,1,1,1,1,1,1,1,1
1,Complete,2018-12-31,1,1,1,1,1,1,1,1,1,1
2,Complete,2019-12-31,2,2,2,2,2,2,2,2,2,2
3,Complete,2020-12-31,67,67,67,67,67,67,67,67,67,67
4,Complete,2021-12-31,399,399,399,399,399,399,399,399,399,399
5,Complete,2022-12-31,783,783,783,783,783,783,783,783,783,783
6,Needed,2016-12-31,1,1,1,1,1,1,1,1,1,1
7,Needed,2017-12-31,4,4,4,4,4,4,4,4,4,4
8,Needed,2018-12-31,9,9,9,9,9,9,9,9,9,9
9,Needed,2019-12-31,124,124,124,124,124,124,124,124,124,124


In [125]:
group = df.groupby(pd.Grouper(key = 'App Date (M/D/Y)', freq = 'Y'))
df2 = group.agg({'App No':'count'}).reset_index()
df2['App Date (Y)'] = df2.iloc[:,0].apply(lambda x: x.year)

,App No
App Date (M/D/Y),
1997-12-31,3
1998-12-31,0
1999-12-31,0
2000-12-31,0
2001-12-31,0
2002-12-31,0
2003-12-31,0
2004-12-31,0
2005-12-31,0


In [101]:
# connected=True means it will download the latest version of plotly javascript library.
plotly.offline.init_notebook_mode(connected = True)
from plotly.graph_objs import Figure, Layout, Histogram, Bar, Scatter

getbins = lambda t:{'start':t.min(),'end':t.max(), 'size':5}

bar0 = Bar(x = df2['App Date (Y)'], y = df2['App No'], text = df2['App Date (Y)'])
data = [bar0]
layout = Layout(title = "Bar of TLC Apps per Y",xaxis_title="Year", yaxis_title="No. of Apps")
fig1 = Figure(data = data, layout = layout)

plotly.offline.iplot(fig1, show_link = False, image_width = 600, image_height = 400)

In [141]:
import plotly.express as px
fig = px.bar(df3, x='App Date (M/D/Y)', y='App No',labels={'App Date (M/D/Y)':'App Date (Y)'},
             color='Driver Exam', barmode='group',
             height=400)
fig.show()

# getbins = lambda t:{'start':t.min(),'end':t.max(), 'size':5}

# bar1 = Bar(x = df3['App Date (M/D/Y)'], y = df2['App No'], text = df2['App Date (Y)'])
# data = [bar0]
# layout = Layout(title = "Bar of TLC Apps per Y",xaxis_title="Year", yaxis_title="No. of Apps")
# fig1 = Figure(data = data, layout = layout)

# plotly.offline.iplot(fig1, show_link = False, image_width = 600, image_height = 400)